## 畳み込みニューラルネットワーク(CNN)

### 7.2.5 3次元データの畳み込み演算
画像の場合、縦・横方向に加えてチャンネル方向も合わせた3次元のデータを扱う必要がある。<br>

![iwaki](../images/329fe6c10d0051d35cfb9eab31a4c023.png)

### 7.4.1 4次元配列
CNNで各層を流れるデータは4次元のデータである。(高さ・横幅・チャンネル数・データ数)

In [14]:
import numpy as np
x = np.random.rand(10,1,28,28)
x.shape

(10, 1, 28, 28)

In [18]:
#print(x[0])
print(x[0].shape)

(1, 28, 28)


畳み込み演算の際に真面目に計算を行うとfor文地獄に陥り、numpyを用いた実装だと遅くなってしまう。<br>
そこで、im2colという、便利な関数を用いて実装する。

In [23]:
import sys,os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1,5,5,stride=1,pad=0)
print(col1.shape)

x2 = np.random.rand(10,3,7,7)
col2 = im2col(x2,5,5,stride=1,pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [26]:
class Convolution:
    def __init__(self,W,b,stride=1,pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    def forward(self,x):
        FN,C,FH,FW = self.W.shape
        N,C,H,W = x.shpae
        out_h = int(1 + (H + 2*self.pad -FH)/self.stride) 
        out_w = int(1 + (W + 2*self.pad -FW)/self.stride) 

        col = im2col(x,FH,FW,self.stride,self.pad)
        col_W = self.W.reshape(FN,-1).T
        out = np.dot(col,col_W) + self.b

        out = out.reshape(N,out_h,out_w,-1).transpose(0,3,1,2)
        return out

reshape(FN,-1),-1を指定することで多次元配列の要素数の辻褄があるように要素数をまとめてくれる。

In [31]:
class Pooling:
    def __inti__(self,pool_h,pool_w,stride=2,pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

    def forward(self,w):
        N,C,H,W = x.shape
        out_h = int(1 + (H - self.pool_h)/self.stride)
        out_w = int(1 + (W - self.pool_w)/self.stride)

        col = im2col(x,self.pool_h,self.pool_w,self.stride,self.pad)
        col = col.reshape(-1,self.pool_h*self.pool_w)

        out = np.max(col,axis=1)
        out = out.reshape(N,out_h,out_w,C).transpose(0,3,1,2)